In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
data_root_path = '/home/ec2-user/Neural_Network/Neural_Network/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test').T

print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs].T
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1).T
y_train = np.delete(y_train,indexs)

{'horse': 7, 'frog': 6, 'dog': 5, 'deer': 4, 'cat': 3, 'ship': 8, 'truck': 9, 'automobile': 1, 'bird': 2, 'airplane': 0}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [10]:
print(X_validation.shape)
print(X_test.shape)
arr = np.arange(X_train.shape[0])
np.random.shuffle(arr)
print(arr)

(5000, 3072)
(10000, 3072)
[44775 39939  4499 ...,  3846 10062 28793]


In [11]:
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        m = X.shape[0]
        start_index = np.random.randint(0, m - batch_size)
        X_batch = X[start_index:(start_index + batch_size), :]
        y_batch = y[start_index:(start_index + batch_size)]
        
        return X_batch, y_batch

In [12]:
logdir = 'tf_logs/summary_resNet1'
iteration = 6000
batch_size = 128
learning_rate = 0.001

In [13]:
# try use a function to create a layer
# def create_conv(filters, input_layer):
#     conv = tf.layers.conv2d(inputs=input_layer, filters=filters, kernel_size=3, strides=1, 
#                                     padding = 'SAME', activation = tf.nn.relu, name='conv')
#     pool = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool')
#     return pool

In [14]:
def save_variable(var, name):
    with tf.name_scope(name) as scope:
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        stddev = tf.sqrt(tf.reduce_mean(var - mean))
        tf.summary.scalar('stddev', stddev)
        tf.summary.histogram('histgram', var)

In [15]:
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
with tf.device('/gpu:0'):
    with tf.name_scope('input') as scope:
        X = tf.placeholder(tf.float32, shape = (None, 3072))
        y = tf.placeholder(tf.int32, shape = (None))
        dropout_rate = tf.placeholder(tf.float32, shape=())
        training = tf.placeholder(tf.bool)
        input_layer = tf.reshape(X, [-1, 32, 32, 3])

    with tf.name_scope('conv1') as scope:
        conv1 = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv1',
                                kernel_regularizer = regularizer)
#         weight1 = tf.contrib.layers.flatten(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1'))
        weight1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1')[0]
        batch_norm1 = tf.layers.batch_normalization(conv1, axis = 1, name='batch_norm1')
        dropout1 = tf.layers.dropout(batch_norm1,rate=0.3, training=training, name='dropout1')
#         pool1 = tf.layers.max_pooling2d(inputs=batch_norm1, pool_size=2, strides=2, padding='SAME', name='pool1')

    with tf.name_scope('conv2') as scope:
        conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv2',
                                kernel_regularizer = regularizer)
        weight2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv2')[0]
        
        combine2 = tf.add(dropout1, conv2, name='combine2')
        batch_norm2 = tf.layers.batch_normalization(combine2, axis = 1, name='batch_norm2')
        pool2 = tf.layers.max_pooling2d(inputs=batch_norm2, pool_size=2, strides=2, padding='SAME', name='pool2')
        
    with tf.name_scope('conv3') as scope:
        conv3 = tf.layers.conv2d(inputs=pool2, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv3',
                                kernel_regularizer = regularizer)
        weight3 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv3')[0]
        
        combine3 = tf.add(pool2, conv3, name='combine3')
        batch_norm3 = tf.layers.batch_normalization(combine3, axis = 1, name='batch_norm3')
        dropout3 = tf.layers.dropout(batch_norm3, rate=0.4, training=training, name='dropout3')
#         pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2, padding='SAME', name='pool3')
        
    with tf.name_scope('conv4') as scope:
        conv4 = tf.layers.conv2d(inputs=dropout3, filters=128, kernel_size=3, strides=2, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv4',
                                kernel_regularizer = regularizer)
        weight4= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv4')[0]
        batch_norm4 = tf.layers.batch_normalization(conv4, axis = 1, name='batch_norm4')
        pool4 = tf.layers.max_pooling2d(inputs=batch_norm4, pool_size=2, strides=2, padding='SAME', name='pool4')

    with tf.name_scope('fc') as scope:
        pool4_flat = tf.reshape(pool4, [-1, 4*4*128], name='pool4_flat')
        dropout_fc1 = tf.layers.dropout(pool4_flat, rate=0.4, training=training, name='dropout_fc1')
        
        dense1 = tf.layers.dense(dropout_fc1, units = 8*128, activation = tf.nn.relu, name='dense1',
                                kernel_regularizer = regularizer)
        batch_norm5 = tf.layers.batch_normalization(dense1, axis = 1, name='batch_norm5')
        dropout_fc2 = tf.layers.dropout(batch_norm5, rate=0.4, training=training, name='dropout_fc2')
        
        dense2 = tf.layers.dense(dropout_fc2, units = 128, activation = tf.nn.relu, name='dense2',
                                kernel_regularizer = regularizer)
        batch_norm6 = tf.layers.batch_normalization(dense2, axis = 1, name='batch_norm6')
        dropout_fc3 = tf.layers.dropout(batch_norm6, rate=0.5, training=training, name='dropout_fc3')

    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(dropout_fc3, units = 10, name='logits',kernel_regularizer = regularizer)
        weight_logits = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'logits')[0]

    with tf.name_scope('loss') as scope:
        softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name = 'softmax')
        loss = tf.reduce_mean(softmax) 

    with tf.name_scope('train') as scope:
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    with tf.name_scope('eval') as scope:
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    
        
    with tf.name_scope('summary') as scope:
        save_variable(weight1,'weight1')
        save_variable(weight2,'weight2')
        save_variable(weight3,'weight3')
        save_variable(weight4,'weight4')
        save_variable(weight_logits,'weight_logits')
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(logdir + '/train', tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logdir + '/test', tf.get_default_graph())

    with tf.name_scope('init') as scope:
        init = tf.global_variables_initializer()
    

In [16]:
print(weight4)

<tf.Variable 'conv4/kernel:0' shape=(3, 3, 64, 128) dtype=float32_ref>


In [18]:
def train(X_train, y_train, X_validation, y_validation, iteration, X_test):

    config = tf.ConfigProto()
    config.allow_soft_placement = True
    
    with tf.Session(config=config) as sess:
        sess.run(init)
        for step in range(iteration):
            X_batch, y_batch = get_batch(X_train, y_train, batch_size)
            sess.run(optimizer, feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
                
            if step % 100 == 0:
                summary, loss_, acc = sess.run([merged, loss, accuracy],
                                               feed_dict={X: X_validation, y:y_validation, dropout_rate:0.3, training:False})
                test_writer.add_summary(summary, step)
                print('###################################')
                print('validation! after '+ str(step) + ' iterations' + 
                              ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
                        
                summary, loss_, acc = sess.run([merged, loss, accuracy], 
                                               feed_dict={X: X_batch, y:y_batch,dropout_rate:0.3, training:False})
                train_writer.add_summary(summary, step)
                        
                print('training! after '+ str(step) + ' iterations' + 
                          ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
            
        y_labels = logits.eval(feed_dict={X: X_test, dropout_rate:0.3, training:False})
    return y_labels.T

In [ ]:
y_predicted = train(X_train, y_train, X_validation, y_validation, iteration, X_test)

###################################
validation! after 0 iterations the loss is 2.30877, the accuracy is 0.1048
training! after 0 iterations the loss is 2.28457, the accuracy is 0.125
###################################
validation! after 100 iterations the loss is 1.83644, the accuracy is 0.3012
training! after 100 iterations the loss is 1.74474, the accuracy is 0.34375
###################################
validation! after 200 iterations the loss is 1.6582, the accuracy is 0.3998
training! after 200 iterations the loss is 1.61061, the accuracy is 0.414062
###################################
validation! after 300 iterations the loss is 1.44702, the accuracy is 0.4852
training! after 300 iterations the loss is 1.37792, the accuracy is 0.539062
###################################
validation! after 400 iterations the loss is 1.30674, the accuracy is 0.5282
training! after 400 iterations the loss is 1.32493, the accuracy is 0.484375
###################################
validation! after 500 i

###################################
validation! after 4300 iterations the loss is 0.611516, the accuracy is 0.7994
training! after 4300 iterations the loss is 0.261465, the accuracy is 0.90625
###################################
validation! after 4400 iterations the loss is 0.620733, the accuracy is 0.7894
training! after 4400 iterations the loss is 0.399063, the accuracy is 0.84375
###################################
validation! after 4500 iterations the loss is 0.611659, the accuracy is 0.7952
training! after 4500 iterations the loss is 0.319188, the accuracy is 0.914062
###################################
validation! after 4600 iterations the loss is 0.594322, the accuracy is 0.8064
training! after 4600 iterations the loss is 0.303874, the accuracy is 0.921875
###################################
validation! after 4700 iterations the loss is 0.612514, the accuracy is 0.7986
training! after 4700 iterations the loss is 0.255526, the accuracy is 0.914062
################################

In [14]:
# save_predictions('ans2-yz3065', y_predicted)

In [15]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans2-yz3065.npy')
print(loaded_y.shape)
loaded_y[:10]

(10, 10000)


array([[ -2.07462335,   5.98183823,   3.44229984, ...,  -4.13653469,
          2.89506269,  -8.66244316],
       [ -1.76037085,  12.56741905,  -4.42907667, ...,  -6.14924908,
          7.25259495, -11.99969006],
       [ -2.21564674,  -7.40246391,   1.54524112, ...,   4.15086126,
          0.539554  ,  -1.36715853],
       ..., 
       [ -1.2539717 , -11.40724182,   0.99441135, ...,   4.50973558,
         -1.79880798,  18.51285934],
       [ -2.6909461 ,  15.32911205,  -0.11396468, ...,  -5.40692949,
         -2.11565709,  -8.85854053],
       [ -1.06548655,   0.87547415,  -2.57215023, ...,  -5.48661661,
          2.51511073, -10.11819172]], dtype=float32)